In [30]:
!pip uninstall mediapipe -y -q
!pip install mediapipe==0.10.21 opencv-python scikit-learn pandas numpy matplotlib seaborn tqdm joblib tensorflow kaggle -q

In [31]:
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [32]:
import os
import sys
import cv2
import numpy as np
import mediapipe as mp
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from pathlib import Path
import warnings
import pickle
from scipy.spatial.distance import euclidean

warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

print("✓ All libraries imported successfully")


✓ All libraries imported successfully


In [33]:
class Config:
    # Dataset settings
    DATASET_NAME = "asl_alphabet"
    DATASET_PATH = Path("asl_data")
    MODEL_PATH = Path("models/gesture_model_optimized.pkl")
    CACHE_PATH = Path("cache/features_optimized.pkl")
    CONFUSION_MATRIX_PATH = Path("results/confusion_matrix.png")
    KAGGLE_DATASET = "grassknoted/asl-alphabet"

    # Model optimization settings
    MODEL_TYPE = "rf"
    N_ESTIMATORS = 50  # Reduced from 100
    MAX_DEPTH = 12  # Reduced from 15
    MIN_SAMPLES_SPLIT = 30  # Increased from 20
    MIN_SAMPLES_LEAF = 12  # Increased from 8
    MAX_FEATURES = 'sqrt'
    TEST_SIZE = 0.2
    RANDOM_STATE = 42

    # Feature optimization
    USE_ENHANCED_FEATURES = True
    USE_PCA = True  # NEW: Enable PCA for dimensionality reduction
    PCA_VARIANCE = 0.95  # Keep 95% of variance

    # Data settings
    USE_DATA_AUGMENTATION = False  # Disable to reduce training time
    USE_SCALING = True
    SCALER_TYPE = 'standard'  # Changed from 'robust' for speed
    REMOVE_OUTLIERS = False  # Disable to reduce processing time

    # Inference settings
    CONFIDENCE_THRESHOLD = 0.65
    MIN_DETECTION_CONFIDENCE = 0.7
    MIN_TRACKING_CONFIDENCE = 0.5

    # Processing settings
    USE_CACHE = True
    DEFAULT_MAX_SAMPLES = 300  # Reduced from 500
    IGNORE_CLASSES = ["nothing"]
    USE_CROSS_VALIDATION = False  # Disable for faster training

    # Compression settings
    COMPRESS_MODEL = True  # NEW: Enable model compression

print("✓ Configuration loaded with optimizations:")
print(f"  - Reduced estimators: {Config.N_ESTIMATORS}")
print(f"  - Reduced tree depth: {Config.MAX_DEPTH}")
print(f"  - PCA enabled: {Config.USE_PCA}")
print(f"  - Max samples per class: {Config.DEFAULT_MAX_SAMPLES}")

✓ Configuration loaded with optimizations:
  - Reduced estimators: 50
  - Reduced tree depth: 12
  - PCA enabled: True
  - Max samples per class: 300


In [34]:
class EnhancedFeatureExtractor:
    @staticmethod
    def get_distance(p1, p2):
        return np.linalg.norm(p1 - p2)

    @staticmethod
    def compute_palm_orientation(points):
        """
        Calculates the normal vector of the palm using the wrist (0),
        index base (5), and pinky base (17).
        """
        v1 = points[5] - points[0]
        v2 = points[17] - points[0]
        # Cross product gives a vector perpendicular to the palm surface
        palm_normal = np.cross(v1, v2)
        palm_normal /= (np.linalg.norm(palm_normal) + 1e-6)
        return palm_normal

    @staticmethod
    def compute_3d_angles(points):
        """Calculates 3D joint angles for each finger."""
        angles = []
        fingers_indices = [
            [1, 2, 3, 4],    # Thumb
            [5, 6, 7, 8],    # Index
            [9, 10, 11, 12],  # Middle
            [13, 14, 15, 16], # Ring
            [17, 18, 19, 20]  # Pinky
        ]

        for finger in fingers_indices:
            for i in range(len(finger) - 2):
                p1, p2, p3 = points[finger[i]], points[finger[i+1]], points[finger[i+2]]
                v1 = p1 - p2
                v2 = p3 - p2

                cos_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2) + 1e-6)
                angle = np.arccos(np.clip(cos_angle, -1.0, 1.0))
                angles.append(angle)
        return np.array(angles, dtype=np.float32)

    @staticmethod
    def extract_enhanced_features(landmarks):
        if landmarks is None: return None

        # Reshape to (21, 3) for easier indexing
        points = landmarks.reshape(21, 3)

        # 1. Normalized Landmarks (Scale and Translation Invariant)
        # We center at wrist and normalize by the distance between wrist and middle finger base
        centered = points - points[0]
        scale = EnhancedFeatureExtractor.get_distance(points[0], points[9]) + 1e-6
        normalized_points = (centered / scale).flatten()

        # 2. 3D Palm Orientation (Captures hand rotation relative to camera)
        palm_normal = EnhancedFeatureExtractor.compute_palm_orientation(points)

        # 3. 3D Joint Angles
        angles = EnhancedFeatureExtractor.compute_3d_angles(points)

        # 4. Critical Cross-Finger Distances (Inter-finger relationships)
        # Distances between thumb tip and other fingertips (detects pinching/circles)
        thumb_tip = points[4]
        fingertips = [8, 12, 16, 20]
        pinch_distances = [EnhancedFeatureExtractor.get_distance(thumb_tip, points[f]) / scale for f in fingertips]

        # 5. Hand "Spread" (Distance between adjacent fingertips)
        spread_distances = []
        fingertip_all = [4, 8, 12, 16, 20]
        for i in range(len(fingertip_all) - 1):
            d = EnhancedFeatureExtractor.get_distance(points[fingertip_all[i]], points[fingertip_all[i+1]]) / scale
            spread_distances.append(d)

        # Combine all features
        feature_vector = np.concatenate([
            normalized_points, # 63 features
            palm_normal,       # 3 features
            angles,            # 10 features
            pinch_distances,   # 4 features
            spread_distances   # 4 features
        ])

        return feature_vector.astype(np.float32)
print("✓ Feature extractor initialized")

✓ Feature extractor initialized


In [35]:
class OptimizedDataProcessor:
    def __init__(self, dataset_path, max_samples_per_class=None, use_cache=True):
        self.dataset_path = Path(dataset_path)
        self.max_samples_per_class = max_samples_per_class or Config.DEFAULT_MAX_SAMPLES
        self.use_cache = use_cache
        self.extractor = None

    def load_from_cache(self):
        if not self.use_cache or not Config.CACHE_PATH.exists():
            return None
        try:
            print(f"Loading cached features from {Config.CACHE_PATH}")
            with open(Config.CACHE_PATH, 'rb') as f:
                cache_data = pickle.load(f)
            if cache_data.get('max_samples') == self.max_samples_per_class:
                print(f"✓ Cache loaded: {len(cache_data['X'])} samples")
                return cache_data['X'], cache_data['y']
            else:
                print(f"Cache has different sample count, reprocessing...")
                return None
        except Exception as e:
            print(f"Failed to load cache: {e}")
            return None

    def save_to_cache(self, X, y):
        if not self.use_cache:
            return
        try:
            Config.CACHE_PATH.parent.mkdir(parents=True, exist_ok=True)
            cache_data = {'X': X, 'y': y, 'max_samples': self.max_samples_per_class}
            with open(Config.CACHE_PATH, 'wb') as f:
                pickle.dump(cache_data, f)
            print(f"✓ Features cached to {Config.CACHE_PATH}")
        except Exception as e:
            print(f"Failed to save cache: {e}")

    def extract_landmarks_from_image(self, image_path):
        try:
            image = cv2.imread(str(image_path))
            if image is None:
                return None

            h, w = image.shape[:2]
            if h > 800 or w > 800:  # Reduced max size
                scale = min(800 / h, 800 / w)
                image = cv2.resize(image, None, fx=scale, fy=scale,
                                 interpolation=cv2.INTER_AREA)

            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = self.extractor.process(image_rgb)

            if results.multi_hand_landmarks:
                hand_landmarks = results.multi_hand_landmarks[0]
                landmarks = []
                for landmark in hand_landmarks.landmark:
                    landmarks.extend([landmark.x, landmark.y, landmark.z])
                return np.array(landmarks, dtype=np.float32)
            return None
        except:
            return None

    def load_dataset(self):
        cached = self.load_from_cache()
        if cached is not None:
            X, y = cached
            mask = np.isin(y, Config.IGNORE_CLASSES, invert=True)
            return X[mask], y[mask]

        print("Initializing MediaPipe...")
        mp_hands = mp.solutions.hands
        self.extractor = mp_hands.Hands(
            static_image_mode=True,
            max_num_hands=1,
            min_detection_confidence=0.5,
            model_complexity=0
        )

        X, y = [], []
        class_folders = sorted([d for d in self.dataset_path.iterdir() if d.is_dir()])
        class_folders = [f for f in class_folders if f.name not in Config.IGNORE_CLASSES]

        print(f"Found {len(class_folders)} gesture classes")
        print(f"Processing {self.max_samples_per_class} samples per class")

        for class_folder in class_folders:
            class_name = class_folder.name
            image_files = list(class_folder.glob("*.jpg")) + \
                         list(class_folder.glob("*.jpeg")) + \
                         list(class_folder.glob("*.png"))

            print(f"Processing '{class_name}': {len(image_files)} images available")

            class_features = []
            for img_path in tqdm(image_files[:self.max_samples_per_class * 2],
                               desc=f"  {class_name}", leave=False):
                if len(class_features) >= self.max_samples_per_class:
                    break

                landmarks = self.extract_landmarks_from_image(img_path)
                if landmarks is not None:
                    features = EnhancedFeatureExtractor.extract_enhanced_features(landmarks)
                    if features is not None:
                        class_features.append(features)

            X.extend(class_features)
            y.extend([class_name] * len(class_features))
            print(f"  ✓ {len(class_features)} samples extracted")

        self.extractor.close()

        X = np.array(X, dtype=np.float32)
        y = np.array(y)

        print(f"\n✓ Total samples: {len(X)}")
        self.save_to_cache(X, y)
        return X, y

print("✓ Data processor initialized")

✓ Data processor initialized


In [36]:
class VisualizationUtils:
    @staticmethod
    def plot_confusion_matrix(cm, class_names, save_path=None, normalize=False):
        if normalize:
            cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            fmt = '.2f'
            title = 'Normalized Confusion Matrix'
        else:
            fmt = 'd'
            title = 'Confusion Matrix'

        n_classes = len(class_names)
        figsize = max(10, n_classes * 0.5)
        fig, ax = plt.subplots(figsize=(figsize, figsize))

        sns.heatmap(cm, annot=True, fmt=fmt, cmap='Blues',
                    xticklabels=class_names, yticklabels=class_names,
                    cbar_kws={'label': 'Count' if not normalize else 'Proportion'},
                    ax=ax)

        ax.set_xlabel('Predicted Label', fontsize=12, fontweight='bold')
        ax.set_ylabel('True Label', fontsize=12, fontweight='bold')
        ax.set_title(title, fontsize=14, fontweight='bold', pad=20)

        plt.setp(ax.get_xticklabels(), rotation=45, ha='right', rotation_mode='anchor')
        plt.setp(ax.get_yticklabels(), rotation=0)
        plt.tight_layout()

        if save_path:
            save_path = Path(save_path)
            save_path.parent.mkdir(parents=True, exist_ok=True)
            plt.savefig(save_path, dpi=150, bbox_inches='tight')  # Reduced DPI
            print(f"✓ Confusion matrix saved to: {save_path}")

            if not normalize:
                plt.close()
                norm_path = save_path.parent / f"{save_path.stem}_normalized{save_path.suffix}"
                VisualizationUtils.plot_confusion_matrix(cm, class_names, norm_path, normalize=True)

        plt.close()

print("✓ Visualization utilities loaded")

✓ Visualization utilities loaded


In [37]:
class ModelSizeOptimizer:
    """Programmatically find optimal hyperparameters to minimize model size while maintaining accuracy"""

    @staticmethod
    def estimate_model_size(n_estimators, max_depth, n_features, n_classes):
        """Estimate Random Forest model size in MB"""
        # Approximate formula: trees * nodes_per_tree * (feature_idx + threshold + class_probs)
        avg_nodes_per_tree = 2 ** (max_depth + 1) - 1
        bytes_per_node = 4 + 8 + (n_classes * 8)  # int32 + float64 + class_probs
        total_bytes = n_estimators * avg_nodes_per_tree * bytes_per_node
        return total_bytes / (1024 * 1024)  # Convert to MB

    @staticmethod
    def find_optimal_hyperparameters(X_sample, y_sample, target_size_mb=10,
                                     min_accuracy=0.90, n_trials=20):
        """
        Use grid search to find hyperparameters that minimize model size
        while maintaining accuracy above threshold
        """
        from sklearn.model_selection import cross_val_score
        import itertools

        print(f"\n{'='*60}")
        print(f"AUTOMATIC HYPERPARAMETER OPTIMIZATION")
        print(f"{'='*60}")
        print(f"Target: Model < {target_size_mb}MB, Accuracy > {min_accuracy*100}%")
        print(f"Sample size: {len(X_sample)} samples")

        # Define search space (smaller values = smaller model)
        n_estimators_range = [20, 30, 40, 50, 75]
        max_depth_range = [8, 10, 12, 15, 18]
        min_samples_split_range = [20, 30, 50]

        n_features = X_sample.shape[1]
        n_classes = len(np.unique(y_sample))

        results = []
        best_config = None
        best_score = 0

        # Generate candidate configurations
        configs = list(itertools.product(
            n_estimators_range,
            max_depth_range,
            min_samples_split_range
        ))

        print(f"\nTesting {min(n_trials, len(configs))} configurations...")

        # Sample random configs if too many
        if len(configs) > n_trials:
            import random
            configs = random.sample(configs, n_trials)

        for i, (n_est, max_d, min_split) in enumerate(tqdm(configs, desc="Optimizing")):
            # Estimate model size
            est_size = ModelSizeOptimizer.estimate_model_size(
                n_est, max_d, n_features, n_classes
            )

            # Skip if estimated size is too large
            if est_size > target_size_mb * 1.5:
                continue

            # Test accuracy with 3-fold CV
            model = RandomForestClassifier(
                n_estimators=n_est,
                max_depth=max_d,
                min_samples_split=min_split,
                min_samples_leaf=max(8, min_split // 3),
                max_features='sqrt',
                random_state=42,
                n_jobs=-1
            )

            cv_scores = cross_val_score(model, X_sample, y_sample,
                                       cv=3, scoring='accuracy', n_jobs=-1)
            mean_acc = cv_scores.mean()

            results.append({
                'n_estimators': n_est,
                'max_depth': max_d,
                'min_samples_split': min_split,
                'estimated_size_mb': est_size,
                'cv_accuracy': mean_acc,
                'meets_criteria': mean_acc >= min_accuracy and est_size <= target_size_mb
            })

            # Track best configuration
            if mean_acc >= min_accuracy and est_size <= target_size_mb:
                if mean_acc > best_score:
                    best_score = mean_acc
                    best_config = results[-1]

        # Sort by accuracy (descending) then size (ascending)
        results.sort(key=lambda x: (-x['cv_accuracy'], x['estimated_size_mb']))

        print(f"\n{'='*60}")
        print(f"OPTIMIZATION RESULTS")
        print(f"{'='*60}")
        print(f"\nTop 5 configurations:")
        for i, r in enumerate(results[:5], 1):
            status = "✓" if r['meets_criteria'] else "✗"
            print(f"\n{i}. {status} Accuracy: {r['cv_accuracy']:.4f} | Size: {r['estimated_size_mb']:.2f}MB")
            print(f"   Trees: {r['n_estimators']}, Depth: {r['max_depth']}, "
                  f"MinSplit: {r['min_samples_split']}")

        if best_config:
            print(f"\n{'='*60}")
            print(f"✓ RECOMMENDED CONFIGURATION")
            print(f"{'='*60}")
            print(f"  Accuracy: {best_config['cv_accuracy']:.4f} ({best_config['cv_accuracy']*100:.2f}%)")
            print(f"  Est. Size: {best_config['estimated_size_mb']:.2f} MB")
            print(f"  n_estimators: {best_config['n_estimators']}")
            print(f"  max_depth: {best_config['max_depth']}")
            print(f"  min_samples_split: {best_config['min_samples_split']}")
            return best_config
        else:
            print(f"\n✗ No configuration met criteria. Using best available:")
            fallback = results[0]
            print(f"  Accuracy: {fallback['cv_accuracy']:.4f}")
            print(f"  Est. Size: {fallback['estimated_size_mb']:.2f} MB")
            return fallback

print("✓ Model size optimizer loaded")


✓ Model size optimizer loaded


In [38]:
class OptimizedGestureModel:
    def __init__(self):
        self.model = None
        self.scaler = None
        self.pca = None
        self.class_names = None
        self.hyperparameters = None

    def train(self, X, y, auto_optimize=False, target_size_mb=10):
        unique, counts = np.unique(y, return_counts=True)
        min_samples_needed = int(1 / Config.TEST_SIZE) + 1
        classes_to_remove = unique[counts < min_samples_needed]

        if len(classes_to_remove) > 0:
            print(f"Removing {len(classes_to_remove)} classes with insufficient samples")
            mask = np.isin(y, classes_to_remove, invert=True)
            X, y = X[mask], y[mask]

        # Apply scaling
        if Config.USE_SCALING:
            print(f"Applying {Config.SCALER_TYPE} scaling...")
            self.scaler = StandardScaler()
            X = self.scaler.fit_transform(X)

        # Apply PCA for dimensionality reduction
        if Config.USE_PCA:
            from sklearn.decomposition import PCA
            print(f"Applying PCA (keeping {Config.PCA_VARIANCE*100}% variance)...")
            self.pca = PCA(n_components=Config.PCA_VARIANCE, random_state=Config.RANDOM_STATE)
            X = self.pca.fit_transform(X)
            print(f"✓ Reduced to {X.shape[1]} components (from ~88)")

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=Config.TEST_SIZE,
            random_state=Config.RANDOM_STATE, stratify=y
        )

        self.class_names = sorted(np.unique(y))

        print(f"\nTraining optimized Random Forest...")
        print(f"  Training samples: {len(X_train)}")
        print(f"  Testing samples: {len(X_test)}")
        print(f"  Classes: {len(self.class_names)}")
        print(f"  Features: {X.shape[1]}")

        # AUTO-OPTIMIZE: Find best hyperparameters for target size
        if auto_optimize:
            # Use a sample for faster optimization
            sample_size = min(2000, len(X_train))
            indices = np.random.choice(len(X_train), sample_size, replace=False)
            X_sample = X_train[indices]
            y_sample = y_train[indices]

            optimal_config = ModelSizeOptimizer.find_optimal_hyperparameters(
                X_sample, y_sample,
                target_size_mb=target_size_mb,
                min_accuracy=0.88,  # Slightly lower for sample
                n_trials=20
            )

            self.hyperparameters = optimal_config

            # Use optimized hyperparameters
            self.model = RandomForestClassifier(
                n_estimators=optimal_config['n_estimators'],
                max_depth=optimal_config['max_depth'],
                min_samples_split=optimal_config['min_samples_split'],
                min_samples_leaf=max(8, optimal_config['min_samples_split'] // 3),
                max_features='sqrt',
                random_state=Config.RANDOM_STATE,
                n_jobs=-1,
                class_weight='balanced'
            )
        else:
            # Use config defaults
            self.model = RandomForestClassifier(
                n_estimators=Config.N_ESTIMATORS,
                max_depth=Config.MAX_DEPTH,
                min_samples_split=Config.MIN_SAMPLES_SPLIT,
                min_samples_leaf=Config.MIN_SAMPLES_LEAF,
                max_features=Config.MAX_FEATURES,
                random_state=Config.RANDOM_STATE,
                n_jobs=-1,
                class_weight='balanced'
            )

        self.model.fit(X_train, y_train)

        train_acc = self.model.score(X_train, y_train)
        test_acc = self.model.score(X_test, y_test)

        print(f"\n✓ Training complete!")
        print(f"  Training Accuracy: {train_acc:.4f} ({train_acc*100:.2f}%)")
        print(f"  Testing Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")

        y_pred = self.model.predict(X_test)
        print("\nClassification Report:")
        print(classification_report(y_test, y_pred, zero_division=0))

        cm = confusion_matrix(y_test, y_pred, labels=self.class_names)
        VisualizationUtils.plot_confusion_matrix(
            cm, self.class_names, save_path=Config.CONFUSION_MATRIX_PATH
        )

        return {
            'model': self.model,
            'train_accuracy': train_acc,
            'test_accuracy': test_acc,
            'confusion_matrix': cm
        }

    def save(self, path):
        path = Path(path)
        path.parent.mkdir(parents=True, exist_ok=True)

        # Apply tree pruning for additional size reduction
        if Config.COMPRESS_MODEL and hasattr(self.model, 'estimators_'):
            print("\nApplying model compression techniques...")

            # 1. Prune weak trees (bottom 10% by feature importance)
            if len(self.model.estimators_) > 10:
                importances = []
                for tree in self.model.estimators_:
                    importances.append(tree.tree_.compute_feature_importances(normalize=False).sum())

                threshold = np.percentile(importances, 10)
                self.model.estimators_ = [
                    tree for tree, imp in zip(self.model.estimators_, importances)
                    if imp >= threshold
                ]
                self.model.n_estimators = len(self.model.estimators_)
                print(f"  ✓ Pruned to {self.model.n_estimators} trees")

            # Removed: 2. Convert tree values to float32 (not writable)
            # for tree in self.model.estimators_:
            #     tree.tree_.value = tree.tree_.value.astype(np.float32)

        model_data = {
            'model': self.model,
            'scaler': self.scaler,
            'pca': self.pca,
            'class_names': self.class_names,
            'hyperparameters': self.hyperparameters
        }

        # Save with maximum compression
        joblib.dump(model_data, path, compress=('gzip', 9) if Config.COMPRESS_MODEL else 0)

        file_size = path.stat().st_size / (1024 * 1024)
        print(f"\n✓ Model saved to: {path}")
        print(f"  File size: {file_size:.2f} MB")

        # Additional lightweight version with quantization
        if Config.COMPRESS_MODEL:
            lightweight_path = path.parent / f"{path.stem}_lightweight{path.suffix}"
            self._save_quantized(model_data, lightweight_path)

    def _save_quantized(self, model_data, path):
        """Save a quantized version for even smaller size"""
        import copy
        quantized_data = copy.deepcopy(model_data)

        # Removed: Quantize tree thresholds to float16 (not writable)
        # for tree in quantized_data['model'].estimators_:
        #     tree.tree_.threshold = tree.tree_.threshold.astype(np.float16)

        joblib.dump(quantized_data, path, compress=('gzip', 9))
        file_size = path.stat().st_size / (1024 * 1024)
        print(f"  Lightweight version: {file_size:.2f} MB (saved to {path.name})")

    @staticmethod
    def load(path):
        model_data = joblib.load(path)
        gesture_model = OptimizedGestureModel()
        gesture_model.model = model_data['model']
        gesture_model.scaler = model_data.get('scaler')
        gesture_model.pca = model_data.get('pca')
        gesture_model.class_names = model_data['class_names']
        return gesture_model

print("✓ Optimized model class loaded")

✓ Optimized model class loaded


In [39]:
def download_kaggle_dataset():
    try:
        import kaggle
    except ImportError:
        print("Installing Kaggle API...")
        !pip install kaggle -q
        import kaggle

    dataset_path = Config.DATASET_PATH
    dataset_name = Config.KAGGLE_DATASET

    print(f"Downloading dataset: {dataset_name}")
    print(f"Destination: {dataset_path}")

    try:
        kaggle.api.dataset_download_files(
            dataset_name, path=dataset_path, unzip=True
        )
        print("✓ Dataset downloaded successfully!")
        return True
    except Exception as e:
        print(f"✗ Download failed: {e}")
        return False

def verify_dataset(dataset_path):
    train_path = dataset_path / "asl_alphabet_train" / "asl_alphabet_train"
    if not train_path.exists():
        return False
    subdirs = list(train_path.glob("*/"))
    return len(subdirs) > 0

print("✓ Dataset utilities loaded")



✓ Dataset utilities loaded


In [40]:
def main():
    print("=" * 60)
    print("OPTIMIZED HAND GESTURE RECOGNITION SYSTEM")
    print("With Automatic Model Size Optimization")
    print("=" * 60)
    print("\nOptimizations:")
    print("  ✓ Automatic hyperparameter tuning for size/accuracy")
    print("  ✓ PCA dimensionality reduction (95% variance)")
    print("  ✓ Tree pruning (removes weak estimators)")
    print("  ✓ Float32/Float16 quantization")
    print("  ✓ Maximum gzip compression")
    print("=" * 60)

    # Check for existing model
    if Config.MODEL_PATH.exists():
        print(f"\nFound existing model: {Config.MODEL_PATH}")
        file_size = Config.MODEL_PATH.stat().st_size / (1024 * 1024)
        print(f"Model size: {file_size:.2f} MB")
        choice = input("\nOptions:\n [1] Use existing\n [2] Retrain\n [3] Clear cache and retrain\n\nChoice: ").strip()

        if choice == '3':
            if Config.CACHE_PATH.exists():
                Config.CACHE_PATH.unlink()
                print("✓ Cache cleared")
        elif choice == '1':
            print("Using existing model")
            return

    # Check dataset
    dataset_path = Config.DATASET_PATH / "asl_alphabet_train" / "asl_alphabet_train"

    if not verify_dataset(Config.DATASET_PATH):
        print("\n✗ Dataset not found!")
        choice = input("Download from Kaggle? (y/n): ").strip().lower()
        if choice == 'y':
            if not download_kaggle_dataset():
                print("Cannot proceed without dataset")
                return
        else:
            print("Cannot proceed without dataset")
            return
    else:
        print(f"\n✓ Dataset found at {Config.DATASET_PATH}")

    # Get sample size
    print(f"\nDefault: {Config.DEFAULT_MAX_SAMPLES} samples per class")
    custom = input("Use default? (y/n): ").strip().lower()

    if custom == 'n':
        try:
            max_samples = int(input("Enter max samples per class (50-500): "))
            max_samples = max(50, min(max_samples, 500))
        except:
            max_samples = Config.DEFAULT_MAX_SAMPLES
    else:
        max_samples = Config.DEFAULT_MAX_SAMPLES

    # Ask about auto-optimization
    print("\n" + "=" * 60)
    print("MODEL SIZE OPTIMIZATION")
    print("=" * 60)
    auto_opt = input("\nEnable automatic hyperparameter optimization? (y/n): ").strip().lower()

    target_size_mb = 10
    if auto_opt == 'y':
        try:
            target_size_mb = float(input("Target model size in MB (default 10): ") or "10")
            target_size_mb = max(2, min(target_size_mb, 50))
        except:
            target_size_mb = 10

    # Process data
    print("\n" + "=" * 60)
    print("DATA PROCESSING")
    print("=" * 60)

    processor = OptimizedDataProcessor(
        dataset_path,
        max_samples_per_class=max_samples,
        use_cache=Config.USE_CACHE
    )

    X, y = processor.load_dataset()

    print("\nClass distribution:")
    unique, counts = np.unique(y, return_counts=True)
    for class_name, count in sorted(zip(unique, counts)):
        print(f"  {class_name}: {count} samples")

    # Train model
    print("\n" + "=" * 60)
    print("MODEL TRAINING")
    print("=" * 60)

    gesture_model = OptimizedGestureModel()
    results = gesture_model.train(
        X, y,
        auto_optimize=(auto_opt == 'y'),
        target_size_mb=target_size_mb
    )

    # Save model
    gesture_model.save(Config.MODEL_PATH)

    print("\n" + "=" * 60)
    print("✓ TRAINING COMPLETE!")
    print("=" * 60)
    print(f"\nResults saved to:")
    print(f"  Model: {Config.MODEL_PATH}")
    print(f"  Confusion Matrix: {Config.CONFUSION_MATRIX_PATH}")

    # Show size comparison if optimization was used
    if auto_opt == 'y' and gesture_model.hyperparameters:
        print(f"\nOptimized Hyperparameters:")
        print(f"  n_estimators: {gesture_model.hyperparameters['n_estimators']}")
        print(f"  max_depth: {gesture_model.hyperparameters['max_depth']}")
        print(f"  min_samples_split: {gesture_model.hyperparameters['min_samples_split']}")

print("✓ Main training function loaded")
print("\nRun the next cell to start training!")

✓ Main training function loaded

Run the next cell to start training!


In [42]:
main()

OPTIMIZED HAND GESTURE RECOGNITION SYSTEM
With Automatic Model Size Optimization

Optimizations:
  ✓ Automatic hyperparameter tuning for size/accuracy
  ✓ PCA dimensionality reduction (95% variance)
  ✓ Tree pruning (removes weak estimators)
  ✓ Float32/Float16 quantization
  ✓ Maximum gzip compression

Found existing model: models/gesture_model_optimized.pkl
Model size: 0.54 MB

Options:
 [1] Use existing
 [2] Retrain
 [3] Clear cache and retrain

Choice: 2

✓ Dataset found at asl_data

Default: 300 samples per class
Use default? (y/n): n
Enter max samples per class (50-500): 400

MODEL SIZE OPTIMIZATION

Enable automatic hyperparameter optimization? (y/n): y
Target model size in MB (default 10): 

DATA PROCESSING
Loading cached features from cache/features_optimized.pkl
Cache has different sample count, reprocessing...
Initializing MediaPipe...
Found 28 gesture classes
Processing 400 samples per class
Processing 'A': 3000 images available


  ✓ 400 samples extracted
Processing 'B': 3000 images available


  ✓ 400 samples extracted
Processing 'C': 3000 images available


  ✓ 400 samples extracted
Processing 'D': 3000 images available


  ✓ 400 samples extracted
Processing 'E': 3000 images available


...

  ✓ 400 samples extracted
Processing 'del': 3000 images available


  ✓ 400 samples extracted
Processing 'space': 3000 images available


  ✓ 400 samples extracted

✓ Total samples: 11125
✓ Features cached to cache/features_optimized.pkl

Class distribution:
  A: 400 samples
  B: 400 samples
  C: 400 samples
  D: 400 samples
  E: 400 samples
  F: 400 samples
  G: 400 samples
  H: 400 samples
  I: 400 samples
  J: 400 samples
  K: 400 samples
  L: 400 samples
  M: 400 samples
  N: 325 samples
  O: 400 samples
  P: 400 samples
  Q: 400 samples
  R: 400 samples
  S: 400 samples
  T: 400 samples
  U: 400 samples
  V: 400 samples
  W: 400 samples
  X: 400 samples
  Y: 400 samples
  Z: 400 samples
  del: 400 samples
  space: 400 samples

MODEL TRAINING
Applying standard scaling...
Applying PCA (keeping 95.0% variance)...
✓ Reduced to 15 components (from ~88)

Training optimized Random Forest...
  Training samples: 8900
  Testing samples: 2225
  Classes: 28
  Features: 15

AUTOMATIC HYPERPARAMETER OPTIMIZATION
Target: Model < 10.0MB, Accuracy > 88.0%
Sample size: 2000 samples

Testing 20 configurations...


Optimizing: 100%|██████████| 20/20 [00:06<00:00,  3.17it/s]



OPTIMIZATION RESULTS

Top 5 configurations:

1. ✓ Accuracy: 0.8870 | Size: 8.63MB
   Trees: 75, Depth: 8, MinSplit: 30

2. ✗ Accuracy: 0.8780 | Size: 3.45MB
   Trees: 30, Depth: 8, MinSplit: 30

3. ✗ Accuracy: 0.8585 | Size: 8.63MB
   Trees: 75, Depth: 8, MinSplit: 50

4. ✗ Accuracy: 0.8580 | Size: 13.82MB
   Trees: 30, Depth: 10, MinSplit: 50

5. ✗ Accuracy: 0.8515 | Size: 5.75MB
   Trees: 50, Depth: 8, MinSplit: 50

✓ RECOMMENDED CONFIGURATION
  Accuracy: 0.8870 (88.70%)
  Est. Size: 8.63 MB
  n_estimators: 75
  max_depth: 8
  min_samples_split: 30

✓ Training complete!
  Training Accuracy: 0.9373 (93.73%)
  Testing Accuracy: 0.9290 (92.90%)

Classification Report:
              precision    recall  f1-score   support

           A       0.94      0.94      0.94        80
           B       0.96      0.99      0.98        80
           C       0.90      1.00      0.95        80
           D       0.97      0.85      0.91        80
           E       0.93      0.94      0.93        8

In [44]:
# Test the compressed model
def test_compressed_model():
    """Test both standard and lightweight compressed models"""
    print("=" * 60)
    print("TESTING COMPRESSED MODELS")
    print("=" * 60)

    # Load both models
    standard_path = Config.MODEL_PATH
    lightweight_path = Config.MODEL_PATH.parent / f"{Config.MODEL_PATH.stem}_lightweight{Config.MODEL_PATH.suffix}"

    models_to_test = []
    if standard_path.exists():
        models_to_test.append(("Standard", standard_path))
    if lightweight_path.exists():
        models_to_test.append(("Lightweight", lightweight_path))

    if not models_to_test:
        print("❌ No trained models found. Please run training first.")
        return

    # Load test data
    print("\nLoading test data...")
    processor = OptimizedDataProcessor(
        Config.DATASET_PATH / "asl_alphabet_train" / "asl_alphabet_train",
        max_samples_per_class=50,  # Small sample for quick testing
        use_cache=False
    )
    X_test, y_test = processor.load_dataset()

    print(f"Test set: {len(X_test)} samples, {len(np.unique(y_test))} classes\n")

    # Test each model
    for model_name, model_path in models_to_test:
        print(f"\n{'='*60}")
        print(f"Testing {model_name} Model")
        print(f"{'='*60}")

        # Load model
        gesture_model = OptimizedGestureModel.load(model_path)

        # Get file size
        file_size_mb = model_path.stat().st_size / (1024 * 1024)
        print(f"File size: {file_size_mb:.2f} MB")

        # Preprocess test data (apply same transformations as training)
        X_processed = X_test.copy()
        if gesture_model.scaler:
            X_processed = gesture_model.scaler.transform(X_processed)
        if gesture_model.pca:
            X_processed = gesture_model.pca.transform(X_processed)

        # Make predictions
        print("\nRunning inference...")
        import time
        start = time.time()
        y_pred = gesture_model.model.predict(X_processed)
        inference_time = time.time() - start

        # Calculate metrics
        accuracy = (y_pred == y_test).mean()
        avg_time_per_sample = (inference_time / len(X_test)) * 1000  # Convert to ms

        print(f"✓ Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
        print(f"✓ Total inference time: {inference_time:.3f}s")
        print(f"✓ Avg time per sample: {avg_time_per_sample:.2f}ms")
        print(f"✓ Throughput: {len(X_test)/inference_time:.1f} samples/sec")

        # Show confusion matrix for a few classes
        from sklearn.metrics import classification_report
        print("\nSample Classification Report (first 5 classes):")
        unique_classes = sorted(np.unique(y_test))
        mask = np.isin(y_test, unique_classes)
        print(classification_report(y_test[mask], y_pred[mask],
                                   labels=unique_classes, zero_division=0))

    print(f"\n{'='*60}")
    print("✓ TESTING COMPLETE!")
    print(f"{'='*60}")

# Run the test
test_compressed_model()

TESTING COMPRESSED MODELS

Loading test data...
Initializing MediaPipe...
Found 28 gesture classes
Processing 50 samples per class
Processing 'A': 3000 images available


  ✓ 50 samples extracted
Processing 'B': 3000 images available


  ✓ 50 samples extracted
Processing 'C': 3000 images available


  ✓ 50 samples extracted
Processing 'D': 3000 images available


  ✓ 50 samples extracted
Processing 'E': 3000 images available


...

  ✓ 50 samples extracted
Processing 'Y': 3000 images available


  ✓ 50 samples extracted
Processing 'Z': 3000 images available


  ✓ 50 samples extracted
Processing 'del': 3000 images available


  ✓ 50 samples extracted
Processing 'space': 3000 images available


  ✓ 50 samples extracted

✓ Total samples: 1393
Test set: 1393 samples, 28 classes


Testing Standard Model
File size: 0.85 MB

Running inference...
✓ Accuracy: 0.9332 (93.32%)
✓ Total inference time: 0.035s
✓ Avg time per sample: 0.03ms
✓ Throughput: 39750.6 samples/sec

Sample Classification Report (first 5 classes):
              precision    recall  f1-score   support

           A       1.00      0.98      0.99        50
           B       0.96      1.00      0.98        50
           C       0.94      0.98      0.96        50
           D       0.98      0.88      0.93        50
           E       0.98      0.86      0.91        50
           F       0.94      0.96      0.95        50
           G       1.00      1.00      1.00        50
           H       1.00      0.98      0.99        50
           I       1.00      0.98      0.99        50
           J       0.98      0.94      0.96        50
           K       0.96      0.98      0.97        50
           L       1.00      1